In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from astropy.io import fits
import pandas
from spectral_cube import SpectralCube
import plotly.express as px
import astropy.units as u

import warnings
warnings.filterwarnings('ignore')

In [ ]:
RAMPS_data = pandas.read_csv('/users/hfwest/GBO-REU/Data/RAMPS_clump_catalog_diss_data.txt', sep = '\s+')
RAMPS_data['l'] = np.zeros(shape=len(RAMPS_data)) # Create 'l' and 'b' columns
RAMPS_data['l'][0] = '(deg)'
RAMPS_data['b'] = np.zeros(shape=len(RAMPS_data))
RAMPS_data['b'][0] = '(deg)'

for idx in range(1, len(RAMPS_data)): # For each item, extract longitude and latitude from clump name
    RAMPS_data.at[idx,'l'] = float(RAMPS_data.at[idx, 'ClumpName'][1:7])
    sign = 1
    if RAMPS_data.at[idx, 'ClumpName'][7] == '-':
        sign = -1
    RAMPS_data.at[idx, 'b'] = sign * float(RAMPS_data.at[idx, 'ClumpName'][8::])

# Then remove extraneous columns, including the original clump name
RAMPS_data = RAMPS_data.drop(columns=['ClumpName', 'T_rot', 'log10(N_tot,ba)', 'chi(NH3)', 'p1', 'M_clump', 'L_clump', 'EvolState', 
                         'FILLER', 'FILLER.1', 'FILLER.2', 'T_rot_err','log10(N_tot,ba)_err', 'sigma_err', 
                         'V_LSR_err', 'chi(NH3)_err', 'd1_err', 'FILLER.3', 'M_clump_err', 'L_clump_err', 'FILLER.4'])

In [37]:
RAMPS_data

,theta_maj,theta_min,sigma,V_LSR,d1,l,b
0,(arcmin),(arcmin),(km/s),(km/s),(kpc),(deg),(deg)
1,1.7091411264505836,0.9846699238195086,0.314694522311,5.2894819608,4.8,9.612,0.142
2,3.805428054363068,2.402000572584829,0.83105567403,4.28876439395,4.8,9.623,0.191
3,1.0624365856111189,0.8490050974098039,0.333130734574,2.70899364263,3.86,9.629,0.266
4,0.905427652250561,0.7072311175166667,0.224414712996,22.4674223611,2.81,9.634,0.298
...,...,...,...,...,...,...,...
2676,4.052036432863452,0.9481792718128992,0.861155851189,56.2743968352,4.17,47.044,0.248
2677,1.7010196911251547,0.923016748643936,0.420704288902,58.449039996,4.24,47.141,0.319
2678,2.917814832089954,0.7275766000103534,0.764417439979,58.0328150138,4.27,47.171,0.313
2679,0.9988994922785079,0.5005601272218272,0.633378861293,58.3624671728,4.26,47.223,0.318


In [ ]:
_28_5_data = pandas.read_csv('/users/hfwest/GBO-REU/Data/RAMPS_CLF/28_5.txt', sep = '\s+')
_28_5_data['l'] = np.zeros(shape=len(_28_5_data))
_28_5_data['b'] = np.zeros(shape=len(_28_5_data))
_28_5_data['V_LSR'] = np.zeros(shape=len(_28_5_data))

Cube = SpectralCube.read('/home/scratch/hfwest/RAMPS/Data/L28_5_NH3_1-1_cube.fits') # Bring in spectral cube to obtain longitude, latitude, and V_LSR values from pixel values
v, y, x = Cube.world[:, :, :] # Extract wcs
x_eff = x / u.deg # Remove units
y_eff = y / u.deg
v_eff = v / (u.km/u.s)

for idx in range(len(_28_5_data)): # For each clump found, convert pixel value to real value:
    _28_5_data.at[idx, 'l'] = x_eff[_28_5_data.at[idx, 'v'], _28_5_data.at[idx, 'y'], _28_5_data.at[idx, 'x']]
    _28_5_data.at[idx, 'b'] = y_eff[_28_5_data.at[idx, 'v'], _28_5_data.at[idx, 'y'], _28_5_data.at[idx, 'x']]
    _28_5_data.at[idx, 'V_LSR'] = v_eff[_28_5_data.at[idx, 'v'], _28_5_data.at[idx, 'y'], _28_5_data.at[idx, 'x']]

_28_5_data = _28_5_data.drop(columns= ['Ncl', 'x', 'y', 'v', 'Tpeak', 'FWHMx', 'FWHMy', 'FWHMv', 'Npix']) # Remove extraneous columns including original pixel values


In [24]:
_28_5_data

,Tpeak,FWHMx,FWHMy,R,FWHMv,Sum(T),Npix,l,b,V_LSR
0,3.58,6.39,3.98,5.67,9.96,1379.87,941,28.202502,-0.045833,95.806555
1,3.55,7.19,8.18,7.69,6.99,2264.15,1523,28.569212,-0.239167,87.839274
2,3.51,6.62,5.47,6.68,11.08,2186.10,1728,28.200835,-0.050833,96.005737
3,3.50,7.20,8.71,8.41,5.72,1878.88,1176,28.569211,-0.235833,87.042545
4,3.35,8.82,10.42,10.30,6.67,3542.97,2728,28.564210,-0.234167,85.648271
...,...,...,...,...,...,...,...,...,...,...
225,0.55,1.06,1.06,0.98,1.65,3.65,7,28.950842,0.100833,72.103893
226,0.55,2.42,2.65,2.03,3.83,28.15,54,28.650842,0.105833,106.363202
227,0.55,1.16,1.17,1.13,1.49,3.70,7,28.399167,0.017500,80.469539
228,0.54,2.43,1.73,1.87,4.11,27.42,53,28.657509,0.104167,105.964838
